리트코드 부분 문자열이 포함된 최소 윈도우: https://leetcode.com/problems/minimum-window-substring/

In [1]:
import collections
import heapq
import functools
import itertools
import re
import sys
import math
import bisect
from typing import *

In [2]:
s = "ADOBECODEBANC"
t = "ABC"

s에서 t에 있는 문자들을 모두 포함하는 최소 크기의 슬라이딩 윈도우를 찾는 문제
O(n)의 제한이 있다.

# 모든 윈도우 크기를 브루트 포스로 탐색

In [3]:
def minWindow(s: str, t: str) -> str:
    def contains(s_substr_lst: List, t_lst: List):
        for t_elem in t_lst:
            if t_elem in s_substr_lst:
                s_substr_lst.remove(t_elem)
            else:
                return False
        return True

    if not s or not t:
        return ''

    window_size = len(t)

    for size in range(window_size, len(s) + 1):
        for left in range(len(s) - size + 1):
            s_substr = s[left:left + size]
            if contains(list(s_substr), list(t)):
                return s_substr
    return ''

In [4]:
minWindow(s, t)

'BANC'

t의 크기부터 시작하여 모든 크기의 윈도우를 탐색하는 방식이다. <br>
O(n^2)의 복잡도를 갖기 때문에 TLE가 뜬다.

# 투 포인터, 슬라이딩 윈도우로 최적화 

In [5]:
def minWindow(s: str, t: str) -> str:
    need = collections.Counter(t)
    missing = len(t)
    left = start = end = 0

    # 오른쪽 포인터 이동
    for right, char in enumerate(s, 1):
        missing -= need[char] > 0
        need[char] -= 1

        # 필요 문자가 0이면 왼쪽 포인터 이동 판단
        if missing == 0:
            while left < right and need[s[left]] < 0:
                need[s[left]] += 1
                left += 1

            if not end or right - left <= end - start:
                start, end = left, right
            need[s[left]] += 1
            missing += 1
            left += 1
    return s[start:end]

In [6]:
minWindow(s, t)

'BANC'

Counter로 t에서 필요한 문자와 그 수를 세준다. <br>
투 포인터를 사용하는데 오른쪽 포인터를 움직여 문자들을 순회하며 문자를 찾으면 Counter로 세둔 것을 하나씩 줄인다. missing은 전체 필요한 문자 수로 이 역시도 줄여줘야 한다. <br>
다 찾았으면 0에 위치한 왼쪽 포인터를 줄일 수 있는지 살펴본다. 불필요한 문자를 가리키고 있다면 포인터를 앞으로 이동시켜 최종적으로 윈도우가 최소 크기가 되게 한다.

# Counter로 좀 더 편리한 풀이 

In [7]:
def minWindow(s: str, t: str) -> str:
    t_count = collections.Counter(t)
    current_count = collections.Counter()

    start = float('-inf')
    end = float('inf')

    left = 0
    # 오른쪽 포인터 이동
    for right, char in enumerate(s, 1):
        current_count[char] += 1

        # AND 연산 결과로 왼쪽 포인터 이동 판단
        while current_count & t_count == t_count:
            if right - left < end - start:
                start, end = left, right
            current_count[s[left]] -= 1
            left += 1

    return s[start: end] if end - start <= len(s) else ''

In [8]:
minWindow(s, t)

'BANC'

missing을 사용하지 않고 Counter의 AND 연산을 사용해 모든 결과가 포함되는지 여부를 확인할 수 있다.